## Aula 04d - Fast Fourier Transform

In [150]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
import time
from cmath import exp, pi
%matplotlib inline

A transformada rápida de Fourier (FFT) é um algoritmo de *divisão e conquista* que divide o arranjo de números em duas partes: índices pares e índices ímpares, até obter o caso trivial. É preciso lembrar que as exponenciais complexas (que podem ser decompostas em seno e cosseno) são periódicas e simétricas, e é dessas duas propriedades que sai a possibilidade de computar a FFT.

Em particular, a partir de $e^{-j \frac{2\pi}{N} x u}$, podemos isolar a parte constante e definir a variável $W = e^{-j \frac{2\pi}{N}}$. Note que $W$ é fixo, pois não depende da amostragem no tempo (controlada por $x$), nem da frequência dos senos e cossenos (controlada por $u$). Assim, se tivermos por exemplo um sinal com 4 observações ($N=4$), então teremos:


In [151]:
N = 4
W = exp(-1j*(2*pi)/N)
print(W)

(6.123233995736766e-17-1j)


Que é independente de $u$ e de $x$. As duas propriedades que utilizaremos são: 
1. periodicidade em u,x: $W_N^{ux} = W_N^{u(N+x)} = W_N^{(u+N)x}$   

2. simetria do complexo conjugado: $W_N^{u(N-x)} = W_N^{-ux} = (W_N^{ux})^*$
   por exemplo é fácil ver que para $x=N$, $W_N^{uN} = e^{-j2\pi u} =1$
   
O passo de **divisão** é feito (conforme dissemos anteriormente), decompondo a transformada em índices pares e ímpares. Escrevendo a transformada em termos de $W$:

$F(u) = \sum_{x=0}^{N-1} f(x)W_N^{ux}$

podemos reescrever os índices pares como $2x$ e os índices ímpares com $2x+1$:

$F(u) = \sum_{x = 0}^{N/2-1} f(2x)W_N^{u(2x)} +  \sum_{x =0}^{N/2-1} f(2x+1)W_N^{u(2x+1)}$

e a seguir:

$F(u) = \sum_{x = 0}^{N/2-1} f(2x)\cdot(W_N^2)^{ux} +  \sum_{x =0}^{N/2-1} f(2x+1)\cdot (W_N^2)^{(2x+1)u}$

e então podemos mover para fora da soma dos termos ímpares o termo independente de $x$ que é o $W_N^u$ obtido da distribuição do $u$:

$F(u) = \sum_{x = 0}^{N/2-1} f(2x)\cdot(W_N^2)^{ux} +  W_N^u \cdot \sum_{x =0}^{N/2-1} f(2x+1)\cdot (W_N^2)^{ux}$

E agora vem o truque! pois:
$W_N^2 = e^{-j\frac{2\pi}{N}2} = e^{-j\frac{2\pi}{N/2}} = W_{N/2}$

o que permite escrever:

$F(u) = \sum_{x = 0}^{N/2-1} f(2x)\cdot W_{N/2}^{ux} +  W_N^u \cdot \sum_{x =0}^{N/2-1} f(2x+1)\cdot W_{N/2}^{ux}$

O primeiro termo é a DFT dos $N/2$ elementos correspondentes aos índices pares, o segundo termo é a DFT dos $N/2$ elementos correspondentes aos índices pares!

Assim podemos decompor a DFT de $N$ elementos, de forma recursiva, em $N/2$ DFTs, e posteriormente combinar os resultados:

$F(u) = F_\text{par}(u) + W_N^u \cdot F_\text{ímpar}(u)$

In [166]:
N = 4
f = [3,4,2,1]
f

[3, 4, 2, 1]

In [167]:
# separamos em partes pares e ímpares
f_par = f[0::2]
f_imp = f[1::2]
print(f_par)
print(f_imp)

[3, 2]
[4, 1]


In [168]:
# separamos a parte par em partes pares e ímpares
f_parpar = f_par[0::2]
f_parimp = f_par[1::2]
print(f_parpar)
print(f_parimp)

[3]
[2]


Nesse exemplo simples, particionamos até chegar no caso base, em que temos 1 elemento par e 1 ímpar e agora conseguimos calcular:

In [169]:
respar0 = f_parpar[0] + exp(-2j*pi*0/N) * f_parimp[0]
respar1 = f_parpar[0] - exp(-2j*pi*0/N) * f_parimp[0]
respar = [respar0, respar1]
print(respar)

[(5+0j), (1+0j)]


Agora esse resultado é guardado e precisamos executar o outro lado da recursão, relativo aos elementos ímpares.


In [170]:
# separamos a parte ímpar em partes pares e ímpares
f_imppar = f_imp[0::2]
f_impimp = f_imp[1::2]
print(f_imppar)
print(f_impimp)

[4]
[1]


In [171]:
resimp0 = f_imppar[0] + exp(-2j*pi*0/N) * f_impimp[0]
resimp1 = f_imppar[0] - exp(-2j*pi*0/N) * f_impimp[0]
resimp = [resimp0, resimp1]
print(resimp)

[(5+0j), (3+0j)]


Agora combinamos os resultados individuais (respar e resimp):

In [172]:
# pela simetria, posso usar o resultado 0 para obter tambem 0+N/2 = N/4 = 2
res0 = respar[0] + exp(-2j*pi*0/N) * resimp[0]
res2 = respar[0] - exp(-2j*pi*0/N) * resimp[0]

# pela simetria, posso usar o resultado 1 para obter tambem 1+N/2 = 1+N/4 = 3
res1 = respar[1] + exp(-2j*pi*1/N) * resimp[1]
res3 = respar[1] - exp(-2j*pi*1/N) * resimp[1]

# compondo em um único vetor
F_manual =  np.array([res0, res1, res2, res3]).astype(np.complex64)
F_manual

array([10.+0.j,  1.-3.j,  0.+0.j,  1.+3.j], dtype=complex64)

Vamos implementar essa ideia em uma função.

In [173]:
def FFT(A):
    N = len(A)
    if N <= 1:
        return A
    
    # divisao
    par   = FFT(A[0::2])
    impar = FFT(A[1::2])

    # combinacao
    temp = np.zeros(N).astype(np.complex64)
    
    # faco apenas para metade das frequencias, 
    # computando de forma simetrica u+N/2
    for u in range(N//2):
        temp[u] = par[u] + exp(-2j*pi*u/N) * impar[u] # conquista
        temp[u+N//2] = par[u] - exp(-2j*pi*u/N)*impar[u]  # conquista
                
    return temp

In [174]:
# testando o resultado
F_fft = FFT(f)
F_fft

array([10.+0.j,  1.-3.j,  0.+0.j,  1.+3.j], dtype=complex64)

In [181]:
# usando a nossa ultima implementacao da DFT
def DFT1D(A):
   
    F = np.zeros(A.shape, dtype=np.complex64)
    N = A.shape[0]

    # criamos os indices para x
    x = np.arange(N)
    # para cada frequencia, computamos de forma vetorial para x e somamos em 'u'
    for u in np.arange(N):
        F[u] = np.sum(A*np.exp(-2j * pi * u*x / N ))
    
    return F

F_dft = DFT1D(np.array(f))
F_dft

array([10.+0.0000000e+00j,  1.-3.0000000e+00j,  0.-3.6739403e-16j,
        1.+3.0000000e+00j], dtype=complex64)

In [182]:
print(F_dft)
print(F_manual)
print(F_fft)

[10.+0.0000000e+00j  1.-3.0000000e+00j  0.-3.6739403e-16j
  1.+3.0000000e+00j]
[10.+0.j  1.-3.j  0.+0.j  1.+3.j]
[10.+0.j  1.-3.j  0.+0.j  1.+3.j]


apenas um dos termos, o terceiro valor, teve uma diferença na parte imaginária num valor $\sim 3.67 \times 10^{-16}$, um erro bastante baixo.

Essa implementação assume $N=2^x$, e existem outras implementações para compensar essa limitação. No entanto, com esse algoritmo já é possível entender como podemos vencer a complexidade quadrática da DFT, obtendo o mesmo resultado em tempo $O(N \log_2{N})$